### Loading the Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

plt.rcParams['font.family'] = 'AppleGothic'  
plt.rcParams['axes.unicode_minus'] = False    

In [2]:
d = {}

for data_name in ['adj_close', 'mc', 'sales', 'op', 'ni', 'liab', 'eq']:
    print(f"read csv {data_name} ...")
    d[data_name] = pd.read_csv(f"stock.{data_name}.csv", index_col=0, encoding='cp949', parse_dates=True)
    print(f"done!  {d[data_name].shape}")

read csv adj_close ...
done!  (2278, 582)
read csv mc ...
done!  (2278, 582)
read csv sales ...
done!  (2278, 582)
read csv op ...
done!  (2278, 582)
read csv ni ...
done!  (2278, 582)
read csv liab ...
done!  (2278, 582)
read csv eq ...
done!  (2278, 582)


### Data Information

* Data file: stock.{table_name}.csv
* Data time frame: 2018-06-01 ~ 2020-10-14 
* Number of stocks: 2,278
* The currency of the data is in Won (South Korea Currency)
* Explanation of each dataset
  * `stock.adj_close.csv`: the adjusted closing price for each stock.
  * `stock.mc.csv`: the market capitalization in millions of Won 
  * `stock.sales.csv`: the sales revenue in millions of Won 
  * `stock.op.csv`: the operating profit in millions of Won
  * `stock.ni.csv`: the net income in millions of Won 
  * `stock.liab.csv`: the liabilities in millions of Won
  * `stock.eq.csv`: the equity in millions of Won
  * `stock.itemname.csv`: the stock code and the item name
  * `stock.sector.csv`: the stock code and the sector